In [ ]:
# Re-execute after code state reset
import asyncio
from playwright.async_api import async_playwright

async def optimized_dropdown_clicker():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False, args=["--disable-dev-shm-usage", "--no-sandbox"])
        page = await browser.new_page()
        await page.goto("https://alison.legislature.state.al.us/code-of-alabama")

        # Zoom out for better visibility
        await page.evaluate("document.body.style.zoom = '0.3'")
        await page.wait_for_timeout(200)  # quick render pause

        clicked_indexes = set()
        scroll_attempts = 0
        max_scroll_attempts = 15

        while scroll_attempts < max_scroll_attempts:
            # Extract div info in one evaluate block
            dropdown_infos = await page.evaluate("""
                () => {
                    return Array.from(document.querySelectorAll("div.flex.flex-col"))
                        .map(el => ({
                            className: el.className,
                            dataIndex: el.getAttribute("data-index"),
                            hasHref: !!el.querySelector("a[href]")
                        }));
                }
            """)

            # Find a click target
            found = False
            dropdown_divs = await page.query_selector_all("div.flex.flex-col")

            for i, info in enumerate(dropdown_infos):
                if "pb-1" in info["className"] or not info["dataIndex"] or info["hasHref"]:
                    continue
                if info["dataIndex"] in clicked_indexes:
                    continue

                try:
                    print(f"[INFO] Clicking element with data-index: {info['dataIndex']}")
                    await dropdown_divs[i].click()
                    clicked_indexes.add(info["dataIndex"])
                    await page.wait_for_timeout(100)
                    found = True
                    scroll_attempts = 0
                    break
                except Exception as e:
                    print(f"[WARN] Failed to click: {e}")

            if not found:
                scrollable_selector = (
                    "div.relative.mt-0\\.5.h-full.overflow-y-auto.contain-strict.md\\:min-h-0"
                )
                scrollable = await page.query_selector(scrollable_selector)
                if scrollable:
                    prev_scroll = await scrollable.evaluate("el => el.scrollTop")
                    await scrollable.evaluate("el => el.scrollTop += 2000")
                    await page.wait_for_timeout(300)
                    new_scroll = await scrollable.evaluate("el => el.scrollTop")

                    if new_scroll == prev_scroll:
                        scroll_attempts += 1
                        print(f"[SCROLL {scroll_attempts}] No scroll movement.")
                    else:
                        scroll_attempts = 0
                        print(f"[SCROLL SUCCESS] Scrolled to: {new_scroll}")
                else:
                    print("[WARN] Scroll container not found.")
                    break

        print("[INFO] Finished clicking all dropdowns.")
        await browser.close()

await optimized_dropdown_clicker()
